# LLL ALGORITHM _ 210331 Hyewon Sung

In [37]:
import sys
import math
import numpy as np
from numpy import linalg as la

### input lattice basis

In [38]:
basis = np.array([[-2., 7., 7., -5.],
                  [3., -2., 6., -1.],
                  [2., -8., -9., -7.],
                  [8., -9., 6., -4.]], dtype = np.float64)
alpha = 1


### Functions about GSO Process
* gram_schmidt() function : orthogonalize the input basis, then output orthogonal basis with coefficient matrix M.

In [39]:
def projection_coefficient(u,v) :
    coef = np.dot(u,v)/np.dot(v,v)
    return coef

def projection(u,v) :
    coef = projection_coefficient(u,v)
    return coef*v

def gram_schmidt() :
    orthobasis = reduced_basis.copy()
    M = np.zeros((len(reduced_basis), len(reduced_basis)))
    M[0][0] = 1
    for i in range(1, reduced_basis.shape[1]) :
        for j in range(0,i) :
            orthobasis[i] = orthobasis[i] - projection(reduced_basis[i], orthobasis[j])
            M[i][j] = projection_coefficient(reduced_basis[i],orthobasis[j])
        M[i][i] = 1
    return orthobasis, M


### Functions about Reduction and Exchange process
* reduce(k, l, M, reduced_basis) : makes kth reduced basis orthogonal to lth reduced basis
* exchange(k, reduced_basis, gamma_star, M) : interchanges vector kth reduced basis and k-1th reduced basis. As basis exchanges, so does the coefficient matrix.

In [42]:
def nearest_integer(k,l) :
    num = M[k-1][l-1]
    if (num - math.floor(num) <= 1/2) :
        return math.floor(num) 
    else :
        return math.ceil(num)

def reduce(k,l, M, reduced_basis, iteration) :
    if abs(M[k-1][l-1]) > 1/2 :
        reduced_basis[k-1] = reduced_basis[k-1] - nearest_integer(k,l)*reduced_basis[l-1]
        iteration = iteration+1
        print("\n Iteration : ", iteration)
        print("<reduce> when k is",k, ":","\n", reduced_basis)
        for i in range(0, l) :
            M[k-1][i] = M[k-1][i] - nearest_integer(k,l)*M[l-1][i]
        M[k-1][l-1] = M[k-1][l-1] - nearest_integer(k,l)
        return reduced_basis, M, iteration
    else :
        print("<reduce> doesn't happen this time.")
        return reduced_basis, M, iteration

            
        
        
def exchange(k, reduced_basis, gamma_star, M, iteration) :
    new = reduced_basis.copy()
    z = new[k-2]
    t = reduced_basis[k-1]

    for i in range(len(reduced_basis)) :
        reduced_basis[k-2][i] = t[i]
    for i in range(len(reduced_basis)) :
        reduced_basis[k-1][i] = z[i]
        
    iteration = iteration+1
    print("\n Iteration : ", iteration)
    print("<exchange> when k is",k, ":", "\n",reduced_basis)
    
    nu = M[k-1][k-2]
    delta = gamma_star[k-1] + (nu**2)*(gamma_star[k-2])
    M[k-1][k-2] = (nu*gamma_star[k-2])/delta
    gamma_star[k-1] = (gamma_star[k-1]*gamma_star[k-2])/delta
    gamma_star[k-2] = delta
    
    for j in range(1, k-1) :
        t = M[k-2][j-1]
        M[k-2][j-1] = M[k-1][j-1]
        M[k-1][j-1] = t
    
    for i in range(k+1, len(reduced_basis)+1) :
        ks = M[i-1][k-1]
        M[i-1][k-1] = M[i-1][k-2] - nu*M[i-1][k-1]
        M[i-1][k-2] = M[k-1][k-2]*M[i-1][k-1] + ks
        
    return reduced_basis, gamma_star, M, iteration
        
        

### Main Fuction

In [43]:
#main loop

reduced_basis = basis.copy()
orthobasis, M = gram_schmidt()

print("input basis : ","\n", basis,"\n")
#print("GSO basis : ",orthobasis)
#print("coefficient matrix : ", M)


gamma_star = np.zeros((len(basis), 1))
for i in range(len(orthobasis)) :
    for j in range(len(orthobasis)) :
        gamma_star[i] += orthobasis[i][j] ** 2

k = 2
iteration = 0

print("Below is the process of each iteration whether it goes to reduction or exchange function and show the change of reduced basis. \n ")
#basis[0] : row 개수, basis[1] : col 개수
while k <= reduced_basis.shape[0] :
    reduced_basis, M, iteration = reduce(k,k-1, M, reduced_basis, iteration)
    if gamma_star[k-1] >= (alpha-(M[k-1][k-2]**2))*gamma_star[k-2] :
        for l in range(k-2, 0, -1) :
            reduced_basis, M, iteration = reduce(k, l, M, reduced_basis, iteration)
        k = k+1
    else :
        reduced_basis, gamma_star, M, iteration = exchange(k, reduced_basis, gamma_star, M, iteration)
        if k>2 :
            k = k-1
            
            
#print("input basis : ","\n", basis,"\n")
print("\n")
print("final output reduced basis : ", "\n",reduced_basis,"\n")
print("final orthobasis : ", "\n", orthobasis, "\n")
print("final coefficient matrix : ", "\n", M, "\n")


print("length of reduced basis :")
length = 0
for i in range(len(reduced_basis)) :
    for j in range(len(reduced_basis)) :
        length += reduced_basis[i][j]**2
    print(length)
    length=0
        

input basis :  
 [[-2.  7.  7. -5.]
 [ 3. -2.  6. -1.]
 [ 2. -8. -9. -7.]
 [ 8. -9.  6. -4.]] 

Below is the process of each iteration whether it goes to reduction or exchange function and show the change of reduced basis. 
 
<reduce> doesn't happen this time.

 Iteration :  1
<exchange> when k is 2 : 
 [[ 3. -2.  6. -1.]
 [-2.  7.  7. -5.]
 [ 2. -8. -9. -7.]
 [ 8. -9.  6. -4.]]

 Iteration :  2
<reduce> when k is 2 : 
 [[ 3. -2.  6. -1.]
 [-5.  9.  1. -4.]
 [ 2. -8. -9. -7.]
 [ 8. -9.  6. -4.]]

 Iteration :  3
<reduce> when k is 3 : 
 [[  3.  -2.   6.  -1.]
 [ -5.   9.   1.  -4.]
 [ -3.   1.  -8. -11.]
 [  8.  -9.   6.  -4.]]

 Iteration :  4
<reduce> when k is 3 : 
 [[  3.  -2.   6.  -1.]
 [ -5.   9.   1.  -4.]
 [  0.  -1.  -2. -12.]
 [  8.  -9.   6.  -4.]]
<reduce> doesn't happen this time.

 Iteration :  5
<exchange> when k is 4 : 
 [[  3.  -2.   6.  -1.]
 [ -5.   9.   1.  -4.]
 [  8.  -9.   6.  -4.]
 [  0.  -1.  -2. -12.]]

 Iteration :  6
<reduce> when k is 3 : 
 [[  3.  -2.   6